# Machine Learning HW.3 

# CNN-Classification

Dataset: https://www.kaggle.com/c/ml2020spring-hw3/data

In [16]:
import os
import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optimz
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,Dataset

In [2]:
def read_photo(path,label = False):
    path_list = os.listdir(path)
    x = np.zeros([len(path_list), 128, 128, 3])
    y = np.zeros([len(path_list), 1])
    for i,f in enumerate(path_list):
        img = cv2.imread(os.path.join(path, f))
        x[i,:,:] = cv2.resize(img, (128, 128))
        if label:
            y[i,:] = int(f.split('_')[0])
    if label:
        return x, y
    else:
        return x

In [6]:
path = '/Users/Juyi/Desktop/Code/Data/food-11'
train_data, train_label = read_photo(os.path.join(path, 'training'), True)
val_data, val_label = read_photo(os.path.join(path, 'validation'), True)
test_data = read_photo(os.path.join(path, 'testing'))

In [4]:
print(train_data.shape,train_label.shape)
print(val_data.shape,val_label.shape)
print(test_data.shape)

(9866, 128, 128, 3) (9866, 1)
(3430, 128, 128, 3) (3430, 1)
(3110, 128, 128, 3)


In [5]:
train_transforms = transforms.Compose([transforms.ToPILImage(), transforms.RandomHorizontalFlip(), 
        transforms.RandomRotation(15), transforms.ToTensor()])
test_transforms = transforms.Compose([transforms.ToPILImage(), transforms.ToTensor()])

class ImgData(Dataset):
    def __init__(self, x, y = None, transforms = None):
        self.x = x
        self.y = y
        if y is not None:
            self.y = torch.LongTensor(y)
        self.transforms = transforms

    def __getitem__(self, index):
        X = self.x[index]
        if self.transforms is not None:
            X = self.transforms(X)
        if self.y is not None:
            Y = self.y[index]
            return X,Y
        else:
            return Y

    def __len__(self):
        return len(self.x)

In [7]:
batch_size = 128
train_set = ImgData(train_data, train_label, train_transforms)
val_set = ImgData(val_data, val_label, test_transforms)
test_set = ImgData(test_data)

train_loader = DataLoader(train_set, batch_size = batch_size, shuffle = True)
val_loader = DataLoader(val_set, batch_size = batch_size, shuffle = False)
test_loader = DataLoader(test_set, batch_size = 128, shuffle = False)

In [13]:
class Cnn(nn.Module):
    def __init__(self):
        super(Cnn, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),      # [64, 128, 128]
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),             # [64, 64, 64]

            nn.Conv2d(64, 128, 3, 1, 1),    # [128, 64, 64]
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),             # [128, 32, 32]

            nn.Conv2d(128, 256, 3, 1, 1),   # [256, 32, 32]
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),             # [256, 16, 16]

            nn.MaxPool2d(256, 512, 3, 1, 1),# [512, 16, 16]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),             # [512, 8, 8]

            nn.MaxPool2d(512, 512, 3, 1, 1),# [512, 8,8]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)              # [512, 4, 4]
        )
        
        self.func = nn.Sequential(
            nn.Linear(512 * 8 * 8, 1024),    # [1024]
            nn.ReLU(),
            nn.Linear(1024, 512),            # [512]
            nn.ReLU(),
            nn.Linear(512, 11)               # [11]
        )
    
    def forward(self, x):
        cnn_out = self.cnn(x)
        cnn_out = cnn_out.view(cnn_out.size()[0], -1)   #flattn
        func_out = self.func(cnn_out)
        return func_out

In [17]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
cnn = Cnn().to(device)
optimzer = optimz.Adam(cnn.parameters(), lr = 0.001)
critization = nn.CrossEntropyLoss()
epoch_num = 2
for epoch in range(epoch_num):
    train_loss = 0
    val_loss = 0
    train_acc = 0
    val_acc = 0
    cnn.train()
    for i,data in enumerate(train_loader):
        optimzer.zero_grad()
        cnn_out = cnn(data[0].to(device))
        batch_loss = critization(cnn_out, data[1].to(device))
        batch_loss.backward()
        optimzer.step()
        break

AttributeError: module 'torch.optim' has no attribute 'adam'